In [2]:
import numpy as np
import matplotlib.pyplot as plt
#import qiskit.quantum_info as qi



#from qiskit import Aer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature import settings
from qiskit_algorithms.optimizers import  SPSA, COBYLA, L_BFGS_B
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit.primitives import Estimator


In [3]:
def reset_intermediate_info():
    global intermediate_info
    intermediate_info.clear()
    print(intermediate_info)
    intermediate_info = {
        'nfev ': [],
        'parameters ': [],
        'energy ': [],
        'stddev ': []
    }

def callback (nfev , parameters , energy , stddev ):
    intermediate_info ['nfev ']. append ( nfev )
    intermediate_info ['parameters ']. append ( parameters )
    intermediate_info ['energy ']. append ( energy )
    intermediate_info ['stddev ']. append ( stddev )

def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

def fermion_to_qubit(problem, second_q_op, mapper_name,  z2symmetry_reduction=None):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()

  qubit_op = mapper.map(second_q_op)

  if z2symmetry_reduction != None:
    mapper = problem.get_tapered_mapper(mapper)
    qubit_op = mapper.map(second_q_op)
  else :
    qubit_op = mapper.map(second_q_op)

  return qubit_op , mapper

intermediate_info = {
'nfev ': [],
'parameters ': [],
'energy ': [],
'stddev ': []
}


basis = 'sto-3g'
settings.use_pauli_sum_op = False

In [4]:
C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

Co = (0,0,0)
O_1 = (C,0,0)
O_2 = (C*np.cos(theta),C*np.sin(theta),0)
Li = (C+L*np.cos(np.pi-theta),-L*np.sin(np.pi-theta),0)

In [38]:
Co_O_dimer_atoms = ["Co","O" ,"O","Li"]
Co_O_dimer_coords = [Co,O_1, O_2, Li]
Co_O_dimer_charge = 0
Co_O_dimer_multiplicity = 1
Co_O_moleculeinfo = MoleculeInfo(Co_O_dimer_atoms, Co_O_dimer_coords, charge=Co_O_dimer_charge, multiplicity=Co_O_dimer_multiplicity)
energy_arr =[]
order=[]
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
#as_transformer = ActiveSpaceTransformer((8,8), 12)
#as_problem = as_transformer.transform(E_problem)

In [39]:
as_num_particles = as_problem.num_particles
as_num_spatial_orbitals = as_problem.num_spatial_orbitals
as_fermionic_hamiltonian = as_problem.hamiltonian
as_second_q_op = as_fermionic_hamiltonian.second_q_op()
qubit_op, mapper = fermion_to_qubit(as_problem, as_second_q_op, "Pa", z2symmetry_reduction=None)
as_init_state = HartreeFock(as_num_spatial_orbitals,as_num_particles,mapper)
uccsd = UCCSD(as_num_spatial_orbitals,as_num_particles,mapper,initial_state=as_init_state, generalized=True)
twolocal = TwoLocal(as_num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=as_init_state)


In [40]:
uccsd.decompose().decompose().depth()

905957

In [42]:
twolocal.decompose().decompose().decompose().decompose().depth()

237

In [5]:
Co_O_dimer_atoms = ["Co","O" ,"O","Li"]
Co_O_dimer_coords = [Co,O_1, O_2, Li]
Co_O_dimer_charge = 0
Co_O_dimer_multiplicity = 1
Co_O_moleculeinfo = MoleculeInfo(Co_O_dimer_atoms, Co_O_dimer_coords, charge=Co_O_dimer_charge, multiplicity=Co_O_dimer_multiplicity)
energy_arr =[]
order=[]
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
as_problem = driver.run()
#num_spatial_orbitals = E_problem.num_spatial_orbitals
#num_particles = E_problem.num_particles
#as_transformer = ActiveSpaceTransformer((8,8), 12)
#as_problem = as_transformer.transform(E_problem)

as_num_particles = as_problem.num_particles
as_num_spatial_orbitals = as_problem.num_spatial_orbitals
as_fermionic_hamiltonian = as_problem.hamiltonian
as_second_q_op = as_fermionic_hamiltonian.second_q_op()
qubit_op, mapper = fermion_to_qubit(as_problem, as_second_q_op, "Pa", z2symmetry_reduction=None)
as_init_state = HartreeFock(as_num_spatial_orbitals,as_num_particles,mapper)
uccsd = UCCSD(as_num_spatial_orbitals,as_num_particles,mapper,initial_state=as_init_state, generalized=True)
twolocal = TwoLocal(as_num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=as_init_state)

print(uccsd.decompose().decompose().depth())
print(twolocal.decompose().decompose().decompose().decompose().depth())

: 

: 